In [14]:
!pip install torch torchtext torchvision

    100% |████████████████████████████████| 61kB 2.2MB/s 
    100% |████████████████████████████████| 2.0MB 7.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torchtext.vocab import GloVe
import torch.nn.functional as f

In [0]:
class BaselineQANN(nn.Module):
  def __init__(self,emb_dim,dict_size,hidden_dim,ctx_window):
    super(BaselineQANN,self).__init__()
    self.emb = nn.Embedding(dict_size,emb_dim) # with learning, try the GloVe without learning
    self.convolution = nn.Conv2d(emb_dim,hidden_dim,kernel_size=(emb_dim,ctx_window))
    self.h_pool = lambda t : self.horizontal_pooling(t)
    
  def forward(self,q,a):
    q_embed = self.emb(q)
    a_embed = self.emb(a)
    q_enc = F.relu(self.conv(q_embed))
    a_enc = F.relu(self.conv(a_embed))
    r_q = self.h_pool(q_enc)
    r_a = self.h_pool(a_enc)
    return f.cosine_similarity(r_q,r_a)
  
  # given a matrix, does a maxpool operation on the rows
  # t.view(t.size(0),-1) = flatten into a 1D tensor each outcome for each input sample
  # passing from (n,a,b) to (n,a) dimensions
  def horizontal_pooling(t):
    return f.max_pool1d(t,t.size(2)).view(t.size(0),-1)
    
  

In [66]:
model = BaselineQANN(10,400,3,5)


BaselineQANN(
  (emb): Embedding(400, 10)
  (convolution): Conv2d(10, 3, kernel_size=(10, 5), stride=(1, 1))
)



---

*Draft snippets*

---

In [64]:
import numpy as np

a = np.array([[[1,2,3],[4,5,6],[7,8,9]],[[10,11,12],[13,14,15],[16,17,18]]])
a = np.random.rand(5,3,2)
print(a)
t = torch.from_numpy(a).double()
#m = nn.MaxPool2d((1,3))
po = f.max_pool1d(t,t.size(2))
#torch.cat(tuple(po),dim=1)
po.view(t.size(0),-1)

[[[0.69551586 0.72831495]
  [0.97549106 0.44400454]
  [0.81144809 0.78331591]]

 [[0.59097798 0.04503149]
  [0.13219984 0.92665085]
  [0.01887362 0.70750769]]

 [[0.08517405 0.55658162]
  [0.81597408 0.91718316]
  [0.04771517 0.619525  ]]

 [[0.140437   0.52938635]
  [0.41542393 0.5476366 ]
  [0.03496714 0.63203588]]

 [[0.52249022 0.3723957 ]
  [0.05802499 0.17635221]
  [0.62788537 0.59941839]]]


tensor([[0.7283, 0.9755, 0.8114],
        [0.5910, 0.9267, 0.7075],
        [0.5566, 0.9172, 0.6195],
        [0.5294, 0.5476, 0.6320],
        [0.5225, 0.1764, 0.6279]], dtype=torch.float64)

In [19]:
from torchvision import datasets,transforms
data = datasets.MNIST('.',train=True,download=True,transform= transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))]))
m(data[0][0])

Processing...
Done!


tensor([[[-0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
          -0.5000, -0.5000],
         [-0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
          -0.5000, -0.5000],
         [-0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
          -0.5000, -0.5000],
         [-0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
          -0.5000, -0.5000],
         [-0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000, -0.5000,
          -0.5000, -0.5000],
         [-0.5000, -0.5000, -0.5000, -0.5000, -0.4294,  0.0333,  0.1863,
           0.5000, -0.5000],
         [-0.5000, -0.5000, -0.3824,  0.1039,  0.4922,  0.4922,  0.4922,
           0.4490, -0.5000],
         [-0.5000, -0.5000,  0.4333,  0.4922,  0.4922,  0.4922, -0.1353,
          -0.2804, -0.5000],
         [-0.5000, -0.5000,  0.3588,  0.4922,  0.4922,  0.4686, -0.5000,
          -0.5000, -0.5000],
         [-0.5000, -0.5000, -0.1863,  0.4922,  0.4922,  0.1039, -0.5000,
         

In [25]:
t.size(3)

RuntimeError: ignored